# I-Data Cleaning

This is the previous step to the analysis of the shark attacks database.


In this notebook we:
* import the messy data set Shark Attack
* prepare it to be analyzed
* export it as a clean CSV data file

We will need the following libraries for this data cleaning task:

In [603]:
import numpy as np
import pandas as pd
import re
import collections
from collections import Counter
from datetime import datetime

In [604]:
db = pd.read_csv('attacks.csv', encoding='cp1252')

### DataBase overview

In [605]:
len(db)

25723

In [606]:
db.columns 

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [607]:
len(db.columns )

24

In [608]:
db.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


### 

## Keeping only relevant data

### Managing Nulls
First of all, we wish to get rid of the missing data. Using ``.dropna()``, we remove all the missing values, reducing considerably the size of the sample.

For simplicity and performance reasons, the columns ``'Unnamed: 22'`` and ``'Unnamed: 23'`` will no longer be taken into consideration.

In [609]:
db = db.iloc[:,:-2]
db

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The rows filled with `NaN` values are very numerous, we don't need them either.

In [610]:
print('Number of items PRE-cleaning:', len(db))
db = db.dropna(how='all') 
print('Number of items POST-cleaning:', len(db))

Number of items PRE-cleaning: 25723
Number of items POST-cleaning: 8703


Looking closely, many empty-filled rows didn't disapear in the previous step. In these cases, ``'Case Number'`` column contains the value '0'.

In [611]:
db = db[db['Case Number'] != '0']
db = db[db['Case Number'] != 'xx']
print('Number of items POST-POST-cleaning:', len(db))

Number of items POST-POST-cleaning: 6302


Columns ``'href formula'``, ``'href'``, ``'pdf'`` contain information but we will not be needing it for our study 

In [612]:
columns_to_drop = ['href formula', 'href', 'pdf']
db.drop(columns=columns_to_drop, inplace=True)
db

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,Case Number.1,Case Number.2,original order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, p. 234",ND.0005,ND.0005,6.0
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND.0004,ND.0004,5.0
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND.0003,ND.0003,4.0
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",ND.0002,ND.0002,3.0


## 

## Age of the victims

Now that the DataFrame is more compact, we can start working with individual series.

In [613]:
ages = db['Age'].dropna()

From the 'collections' library, ``Counter`` will store the elements of the given serie as dictionnary keys and keep a count of the number of occurrences of any value in the container.

In [614]:
Counter(ages)

Counter({'57': 16,
         '11': 37,
         '48': 28,
         '18': 150,
         '52': 35,
         '15': 139,
         '12': 73,
         '32': 69,
         '10': 56,
         '21': 119,
         '34': 50,
         '30': 76,
         '60': 16,
         '33': 44,
         '29': 78,
         '54': 17,
         '41': 38,
         '37': 42,
         '56': 10,
         '19': 142,
         '25': 108,
         '69': 10,
         '38': 48,
         '55': 23,
         '35': 69,
         '46': 27,
         '45': 32,
         '14': 101,
         '40s': 3,
         '28': 80,
         '20': 141,
         '24': 106,
         '26': 83,
         '49': 28,
         '22': 117,
         '7': 22,
         '31': 52,
         '17': 154,
         '40': 56,
         '13': 94,
         '42': 38,
         '3': 5,
         '8': 28,
         '50': 33,
         '16': 138,
         '82': 1,
         '73': 3,
         '20s': 7,
         '68': 7,
         '51': 20,
         '39': 38,
         '58': 15,
        

In order to deal with imprecise information we assume:
* 'Teens', 'teens' and 'young' are 15 years old
* 'x months' refers to babies that are around 1 year old
* People on their '40s', '50s'... are 45, 55 years old respectively
* People on their 'mid-30s' are 35 years old

In [615]:
def correct_age(ages):
    try:
        return int(ages)
    except:
        ages = str(ages)
        
        if ages.upper() in ["TEEN", "TEENS", "YOUNG"]:
            return 15
        
        elif 'months' in ages.split():
            return 1
        
        elif re.findall('\d+', ages):
            return int(np.array(re.findall('\d+', ages)).astype(int).mean())
        
        else:
            return np.nan     

In [616]:
db['Age'] = db['Age'].apply(correct_age)
db['Age'].describe()

count    3458.000000
mean       27.250723
std        13.886661
min         1.000000
25%        17.000000
50%        24.000000
75%        35.000000
max        87.000000
Name: Age, dtype: float64

## 

## Gender of the victims

In [617]:
sex = db['Sex ']
Counter(sex)

Counter({'F': 637, 'M': 5094, nan: 565, 'M ': 2, 'lli': 1, 'N': 2, '.': 1})

Interpreting 'N' as a typo for 'M' (very proximate in the keyboard), we proceed to it's rectification

In [618]:
def correct_sex(sex):
    sex = str(sex).strip()
    if sex == 'N':
        return 'M'
    elif sex in ['M', 'F']:
        return sex
    else:
        return np.nan

In [619]:
db['Sex '] = db['Sex '].apply(correct_sex)
Counter(db['Sex '])

Counter({'F': 637, 'M': 5098, nan: 567})

## 

## Date of the attack

We noticed that the 'Case Number' column corresponds to the date of the attack. 
Since the 'Date' column comes in many different formats and we wish to keep as many values as possible, we will use the 'Case Number' column, which we find to be much more accurate than 'Date'.

In [620]:
Counter(db['Case Number'])

Counter({'2018.06.25': 1,
         '2018.06.18': 1,
         '2018.06.09': 1,
         '2018.06.08': 1,
         '2018.06.04': 1,
         '2018.06.03.b': 1,
         '2018.06.03.a': 1,
         '2018.05.27': 1,
         '2018.05.26.b': 1,
         '2018.05.26.a': 1,
         '2018.05.24': 1,
         '2018.05.21': 1,
         '2018.05.13.b': 1,
         '2018.05.13.a': 1,
         '2018.05.00': 1,
         '2018.05.12': 1,
         '2018.05.09': 1,
         '2018.04.30.R': 1,
         '2018.04.28.b': 1,
         '2018.04.28.a': 1,
         '2018.04.25.b': 1,
         '2018.04.25.a': 1,
         '2018.04.24': 1,
         '2018.04.23': 1,
         '2018.04.22': 1,
         '2018.04.19': 1,
         '2018.04.15.d': 1,
         '2018.04.15.c': 1,
         '2018.04.15.b': 1,
         '2018.04.15.a': 1,
         '2018.04.14': 1,
         '2018.04.10.R': 1,
         '2018.04.09': 1,
         '2018.04.05': 1,
         '2018.04.03': 1,
         '2018.03.31': 1,
         '2018.03.14': 1,
      

In order to manipulate the 'Case Number' column without giving up it's content, we duplicate this column under the name 'date'

In [621]:
db['date'] = db['Case Number']
date = db['date']

We use the imported ``'datetime'`` library. 

In [622]:
def correct_date(date):
    for x in ('%Y-%m-%d', '%Y.%m.%d', '%d/%m/%Y'):
        try:
            return datetime.strptime(date, x)
        except:
            if len(str(date))==12:
                return date[:-2]
            else:
                return np.nan

In [623]:
db['date'] = db['date'].apply(correct_date)
Counter(db['date'])

Counter({nan: 4234,
         '2018.06.03': 2,
         '2018.05.26': 2,
         '2018.05.13': 2,
         '2018.04.30': 1,
         '2018.04.28': 2,
         '2018.04.25': 2,
         '2018.04.15': 4,
         '2018.04.10': 1,
         '2018.03.09': 2,
         '2017.11.30': 2,
         '2017.11.25': 1,
         '2017.11.13': 1,
         '2017.10.31': 1,
         '2017.10.23': 3,
         '2017.09.25': 2,
         '2017.09.24': 2,
         '2017.09.16': 2,
         '2017.09.15': 2,
         '2017.09.10': 2,
         '2017.09.06': 1,
         '2017.09.03': 2,
         '2017.09.02': 3,
         '2017.08.26': 2,
         '2017.07.29': 2,
         '2017.07.26': 1,
         '2017.07.23': 2,
         '2017.07.20': 3,
         '2017.07.14': 2,
         '2017.07.07': 1,
         '2017.06.18': 2,
         '2017.06.14': 1,
         '2017.06.10': 2,
         '2017.06.07': 1,
         '2017.05.06': 1,
         '2017.04.29': 4,
         '2017.04.17': 2,
         '2017.04.12': 2,
         '2017.04.

Ok, but we need a better solution. We can divide the data in 'date' into different columns.

In [624]:
pd.to_datetime(db['date'])

ParserError: month must be in 1..12: 2014.00.00

In [ ]:

# Create date object in given time format yyyy-mm-dd
my_date = datetime.strptime(str(db['date']), "%Y-%m-%d")

print(my_date)
print('Type: ',type(my_date))

## 

## Location of the attack

In [625]:
Counter(db['Location'])

Counter({'Oceanside, San Diego County': 1,
         'St. Simon Island, Glynn County': 1,
         'Habush, Oahu': 1,
         'Arrawarra Headland': 1,
         'La Ticla': 1,
         'Flat Rock, Ballina': 1,
         'Piedade Beach, Recife': 4,
         'Lighhouse Point Park, Ponce Inlet, Volusia County': 1,
         'Cocoa Beach, Brevard  County': 11,
         'Daytona Beach, Volusia County': 30,
         'Cairns Aquarium': 1,
         'Isle of Palms,  Charleston County': 1,
         'Hilton Head Island, Beaufort County': 4,
         "Off Land's End": 1,
         'Dugong Bay': 1,
         'Nahoon Beach, East London': 4,
         'Sharpes Beach, Ballina': 3,
         'Sai Noi Beach': 2,
         'Manuelita': 3,
         'Gearys Beach': 1,
         'Lennox Head': 4,
         'Surfers Point, Prevelly': 1,
         'South Point, Gracetown': 2,
         'Madoogali': 1,
         'Robberg Beach, Plettenberg Bay': 2,
         'Shipwreck’s Beach, Keoneloa Bay, Kauai': 1,
         'Piedade': 1

le paso una lista con todas las ciudades con costa del mundo, si esta ahi se queda. JAJAAJAJA

## 

# -Free Project-

## Fatality of the attack 

In [626]:
fatal = db['Fatal (Y/N)']

In [627]:
Counter(db['Fatal (Y/N)'])

Counter({'N': 4293,
         'Y': 1388,
         nan: 539,
         'M': 1,
         'UNKNOWN': 71,
         '2017': 1,
         ' N': 7,
         'N ': 1,
         'y': 1})

We interpret:
* 'y' is really 'Y'
* 'M' is a typo for 'N' (proximity on keyboard)
* ' N' and 'N ' are really 'N'

In [628]:
def correct_fatality(fatal):
    fatal = str(fatal).strip()
    if fatal == 'M':
        return 'N'
    elif fatal in ['N', 'Y']:
        return fatal
    else:
        return np.nan

In [629]:
db['Fatal (Y/N)'] = db['Fatal (Y/N)'].apply(correct_fatality)

In [630]:
Counter(db['Fatal (Y/N)'])

Counter({'N': 4302, 'Y': 1388, nan: 612})

## 

## Type of accident

In [631]:
Counter(db['Type'])

Counter({'Boating': 203,
         'Unprovoked': 4595,
         'Invalid': 547,
         'Provoked': 574,
         'Questionable': 2,
         'Sea Disaster': 239,
         nan: 4,
         'Boat': 137,
         'Boatomg': 1})

In [632]:
type_ = db['Type']

What we are really interested about here is finding out weather the attack was provoked or unprovoked.

For that reason, from now on we assign: 
* 'Boatomg', 'Boating' and 'Boat' are provoked types of attack
* 'Sea Disaster' are unprovoked types of attack
* 'Questionable' are provoked types of attack
* 'Invalid' are NaN values

In [633]:
def correct_type(type_):
    if type_ in ['Boatomg', 'Boating', 'Boat', 'Questionable']:
        return 'Provoked'
    elif type_ == 'Sea Disaster':
        return 'Unprovoked'
    else:
        return np.nan

In [634]:
db['Type'] = db['Type'].apply(correct_type)

In [635]:
Counter(db['Type'])

Counter({'Provoked': 343, nan: 5720, 'Unprovoked': 239})

## 

### Saving the clean dataset 

In [636]:
db.to_csv('mas_limpio_que_don_limpio.csv')